In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interpolate
import scipy.integrate as integrate
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default="plotly_white"
# Uncomment the following line if plots do not show up in your notebook.
pio.renderers.default = 'iframe'

# Solving Mechanical System Numerically, Phase Space for a mechanical system.

In this notebook we will use the Lagrangian method to find the equation of motion for the pendulum and for the bead on a loop, and then solve the equation of motion numerically. 

In the last step we make a contour plot of the energy of the system and show how the lines of constant energy are the same as the phase space paths taken by the system. 

## Pendulum

The useful coordinate is $\theta$. With the potential reference $U=0$ when the pendulum is straight down, the potential of the pendulum becomes:
$$U_{pendulum}(\theta) = m g l (1 - \cos \theta) $$

The kinetic energy is $T = \frac{1}{2} m l^2 \dot \theta^2$. This gives a Lagrangian:
$$\mathcal{L} = T -U =  \frac{1}{2} m l^2 \dot \theta^2 - m g l (1 - \cos \theta)$$
and the Lagrange equations become:
$$
\begin{array}{lcl}
p_{\theta} &=& \frac{\partial}{\partial \dot \theta} L = m l^2 \dot \theta \\
\dot p_{\theta} &=& \frac{\partial}{\partial \theta} L = -m g l \sin \theta
\end{array}
$$
So for the equation of motion we have:
$$
\frac{d}{dt} p_{\theta} - \dot p_{\theta} = 0 \Rightarrow m l^2 \ddot \theta + m g l \sin \theta = 0
$$
Which leads to the familiar equation:
$$
\ddot \theta + \frac{g}{l} \sin \theta = 0
$$
To solve this numerically, we split this into two first order coupled differential equations:
$$
\begin{array}{lcl}
\omega &=& \frac{d}{dt}\theta \\
\frac{d}{d t} \omega &=& -\frac{g}{l} \sin \theta
\end{array}
$$
This can now be integrated with `odeint`.

## Bead on a Loop

In the stationary frame, the bead on a loop with radius $R$ has the same potential as the pendulum with $l=R$. You can look at the bead on a loop problem as a 3-dimensional pendulum, with a mass on a weightless rod, that is forced to rotate around $\phi$ at a rate of $\omega_0$. That way we still only have $\theta$ as our free parameter. The potential thus remains:

$$U_{pendulum}(\theta) = m g R (1-\cos \theta)$$

The kinetic energy term now needs to account for the rotation as well as the swinging. We can then replace $\dot \phi = \omega_0$, the forced rotation, giving:

$$T = \frac{1}{2} m R^2 (\dot \theta^2 + \omega_0^2 \sin^2 \theta) $$

The Lagrangian is then:

$$\mathcal{L} = T -U =  \frac{1}{2} m R^2 \dot \theta^2 + \frac{1}{2} m R^2 \omega_0^2 \sin^2 \theta  - m g R (1 - \cos \theta)$$

If we now look at the system from the rotating frame, being only concerned about the movement of $\theta$, and
defining $\gamma = \sqrt{g/(\omega_0^2 R)}$ we can write this as an effective potential:

$$
\begin{array}{lcl}
U_{bead} &=&  -\frac{1}{2} m R^2 \omega_0^2 \sin^2 \theta + m g R (1 - cos \theta)) \Rightarrow  \\
U_{bead} &=& m R^2 \omega_0^2 ( -\frac{1}{2} \sin^2 \theta + \gamma^2 (1 - cos \theta)) 
\end{array}
$$

The Lagrange equations now become:
$$
\begin{array}{lcl}
p_{\theta} &=& \frac{\partial}{\partial \dot \theta} L = m R^2 \dot \theta \\
\dot p_{\theta} &=& \frac{\partial}{\partial \theta} L = m R^2 \omega_0^2\sin \theta \cos \theta - m g R \sin \theta
\end{array}
$$
So for the equation of motion we have:
$$
\frac{d}{dt} p_{\theta} - \dot p_{\theta} = 0 \Rightarrow m R^2 \ddot \theta - m R^2 \omega_0^2\sin \theta \cos \theta + m g R \sin \theta = 0
$$
Which leads to the equation of motion:
$$
\begin{array}{lcl}
\ddot \theta &=& \omega_0^2\sin \theta \cos \theta - \frac{g}{R} \sin \theta \Rightarrow \\
\ddot \theta &=& \omega_0^2\sin \theta ( \cos \theta - \gamma^2 \sin \theta) 
\end{array}
$$
To solve this numerically, we split this into two first order coupled differential equations:
$$
\begin{array}{lcl}
\omega &=& \frac{d}{dt}\theta \\
\frac{d}{d t} \omega &=& \omega_0^2\sin \theta ( \cos \theta - \gamma^2 \sin \theta) \\
\frac{d}{d t} \omega &=& \omega_0^2\sin \theta \cos \theta - \frac{g}{R} \sin \theta 
\end{array}
$$
The last version allows for $\omega_0 = 0$ more clearly, which then should give the same solutions as the normal pendulum.
This result can now again be integrated with `odeint`.

In [2]:
def U_pendulum(theta, param):
    """Potential function for the pendulum, parameters are (m, l, g)"""
    m, l, g = param
    return m*g*l*(1 - np.cos(theta))

def U_bead(theta, param):
    """Potential function for the bead on a loop, parameters are (m,R,omega0, g)"""
    m, R, omega0, g = param
    return -1/2*m*R**2*omega0**2*np.sin(theta)**2 + m*g*R*(1-np.cos(theta))

def pendulum(y,t,param):
    """Derivatives for the pendulum, parameters are (m, l, g)"""
    theta,omega = y # Unpack the input for easy of use.
    m, L,g = param  # Note that m is not needed here, but this way param is the same as for U_pendulum.
    d_theta = omega      # Equation one
    d_omega = -g/L * np.sin(theta) # Equation 2.
    return([d_theta,d_omega]) # Return both functions as a list

def bead(y,t,param):
    """Derivatives for the bead on a loop, parameters are (m, R, omega0, g)"""
    theta,omega = y # Unpack the input for easy of use.
    m, R, omega0, g = param
    d_theta = omega      # Equation one
    d_omega = omega0**2*np.sin(theta)*np.cos(theta)-g/R * np.sin(theta) # Equation 2.
    return([d_theta,d_omega]) # Return both functions as a list

In [3]:
# Define the parameters
m = 1
l = 1
g = 1
R = 1
omega0 = 1.5   # since omega = d theta/ d t
param_pendulum = (m, l, g)
param_bead = (m, R, omega0, g)

In [4]:
# Draw the potentials for different values of omega0, where omega0=0 gives the pendulum.
theta_v =  np.linspace(-np.pi, np.pi, 1000)

fig = make_subplots()  # (specs=[[{"secondary_y": True}]])
fig.update_layout(width=1000, height=800,
                  title=go.layout.Title(text=f"Bead on Loop Potentials", x=0.5, xanchor="center"),
                  titlefont=dict(size=22),
                 xaxis_title=r"$\theta$", yaxis_title=r"$U$")

fig.add_trace(go.Scatter(x=theta_v, y=U_pendulum(theta_v,param_pendulum), name="pendulum", line=dict(color="red", width=2)))

colors = px.colors.sequential.Bluered_r
omega0_list = np.linspace(0.,2.,10)
for i in range(len(omega0_list)):
    param_bead = (m, R, omega0_list[i], g)
    color=px.colors.find_intermediate_color(colors[0],colors[1],i/len(omega0_list),'rgb')
    fig.add_trace(go.Scatter(x=theta_v, y=U_bead(theta_v,param_bead), name=r"$\mathrm{Bead\ }\omega_0="+f"{omega0_list[i]:4.2f}$", line=dict(color=color, width=1)))

fig.write_image("Phys615_HW8_figure1.pdf")   
fig.show()


In [5]:
# Now ask SciPy to solve the diffeq for us.
# Setup the time domain for the solution. 
t_end = 4*np.pi # This is a guess, 2 pi is not enough for omega0=2
N     = 128 
t = np.linspace(0.,t_end,N)

solutions = []
y0=[0, 1.]
for i in range(len(omega0_list)):
    param_bead = (m, R, omega0_list[i], g) 
    Energy = 0.5*param_bead[0]*y0[1]**2 + U_bead(y0[0], param_bead)
    solution = integrate.odeint(bead,y0,t,args=(param_bead,))
    solutionT = np.transpose(solution)
    solutions.append([solutionT,Energy])


In [6]:
fig = make_subplots()  # (specs=[[{"secondary_y": True}]])

fig.update_layout(width=1000, height=800,
                  title=go.layout.Title(text="Bead on a Loop Phase Space", x=0.5, xanchor="center"),
                  titlefont=dict(size=22),
                 xaxis_title=r"theta", yaxis_title=r"omega")

sol = solutions[0][0]
fig.add_trace(go.Scatter(x=sol[0], y=sol[1], name="pendulum", line=dict(color="red", width=3)))
for i in range(1,len(omega0_list)):
    sol = solutions[i][0]
    E = solutions[i][1]
    omega = omega0_list[i]
    color=px.colors.find_intermediate_color(colors[0],colors[1],i/len(solutions),'rgb')
    fig.add_trace(go.Scatter(x=sol[0], y=sol[1], name=rf"$\omega_0={omega:5.3f},\ E={E}$", line=dict(color=color, width=2)))
fig.write_image("Phys615_HW8_figure2.pdf")
fig.show()

Figure 2: The phase space plot for a fixed energy of $E=0.5$ for different values of the hoop rotation speed $\omega_0$

In [7]:
omega0 = 2.
theta = -np.pi
param_bead = (m, R, omega0, g)

# Find the angle where the energy is the minimum.
x_tmp = np.linspace(-np.pi,0,10000)
y_tmp = U_bead(x_tmp, param_bead)
arg_min = np.argmin(y_tmp)
Theta_e_min = x_tmp[arg_min]
E_min = y_tmp[arg_min]

# Inverse energy functions, but these are not needed.
# U_inverse1 = interpolate.interp1d(y_tmp[0:arg_min],x_tmp[0:arg_min],kind="cubic")
# And from the minimum to the mid point, theta=0
# U_inverse2 = interpolate.interp1d(y_tmp[arg_min:],x_tmp[arg_min:],kind="cubic")

energy_list = np.arange(E_min+0.05, 0, 0.3)
energy_list = np.append(energy_list,np.arange(0.,2.,0.3))
energy_list = np.append(energy_list,np.arange(U_bead(-np.pi, param_bead)+0.000000000000001,4., 0.3))
y0_list = []

# Compute the initial theta and omega for the energies.
for i in range(len(energy_list)):
    e = energy_list[i]
    color = px.colors.find_intermediate_color(colors[0],colors[1],i/len(energy_list),'rgb')
    if e == 0. or abs(e - U_bead(-np.pi, param_bead)) < 0.001:
        color = '#00FF00'
    if e < U_bead(-np.pi, param_bead):  # = 2*m*R*g:  # Top of loop energy
        theta = Theta_e_min
    else:
        theta = -np.pi
        
    e_theta = U_bead(theta, param_bead)
    e_omega = e - e_theta
    omega = np.sqrt(2*e_omega/m)
    
    y0_list.append([theta, omega, e, True, color])
    y0_list.append([-theta, -omega, e, False, color])
        

In [8]:
N = 256
solutions = []
for i in range(len(y0_list)):
    t_end = 2*np.pi 
    if y0_list[i][2] == 0.:
        t_end = 5*np.pi
    if abs(y0_list[i][2] - U_bead(-np.pi, param_bead)) < 0.001:
        t_end = 5*np.pi
    t = np.linspace(0.,t_end,N)
    y0= [y0_list[i][0], y0_list[i][1]]
    solution = integrate.odeint(bead,y0,t,args=(param_bead,)) # Returns the answer in an array of (x,y) pairs.
    solutionT = np.transpose(solution)  # Turns the array into an array of two arrays, one for x one for y.
    solutions.append(solutionT)

In [9]:
fig = make_subplots()  # (specs=[[{"secondary_y": True}]])
fig.update_layout(width=1000, height=800,
                  title=go.layout.Title(text=r"$\mathrm{Bead\ on\ a\ Loop\ Phase\ Space\ }\omega_0="+f"{omega0}$", x=0.5, xanchor="center"),
                  titlefont=dict(size=22),
                 xaxis_title=r"$\theta$", yaxis_title=r"$\omega$")

sol = solutions[0][0]
#fig.add_trace(go.Scatter(x=sol[0], y=sol[1], name="pendulum", line=dict(color="red", width=3)))
for i in range(0,len(y0_list)):
    sol = solutions[i]
    E = y0_list[i][2]
    show_leg = y0_list[i][3]
    color = y0_list[i][4]
    
    fig.add_trace(go.Scatter(x=sol[0], y=sol[1], name=rf"$E={E:5.3f}$", showlegend=show_leg, line=dict(color=color, width=2)))

fig.update_xaxes(range=[-np.pi,np.pi])
fig.write_image("Phys615_HW8_figure3.pdf")
fig.show()

Figure 3: The phase space for $\omega_0 = 2$, $\gamma=1/2$. The lines in green are the separatrix trajectories.

# Making an Energy Countour plot.
If you make a contour plot of the energy of the system versus $\theta$ and $\omega$, you end up with the constant energy lines as solutions to the differential equations. There is ***no need to solve the Lagrangian!***.

Of course, if you then also want to highlight specific solutions, you will still need a particular fixed energy solutions where the Lagranguan was solved. Note here though that the solution for a Lagrangian system (or a Hamiltonian system), is actually not different from finding the path in the phase space that has a constant energy.

Below I make the contour plot for the energy of the system, and overlay the separatrix curves from the solutions we found before.

In [10]:
def E_bead(theta_omega, param):
    """Return the total energy for a shape=(2,0) theta and omega at parameters."""
    m, R, omega0, g = param
    omega = theta_omega[1]
    theta = theta_omega[0]
    return 0.5*m*omega*omega + U_bead(theta, param)

In [11]:
E_space_x = np.linspace(-np.pi,np.pi,256)
E_space_y = np.linspace(-np.pi,np.pi,256)
E_space = np.meshgrid( E_space_x, E_space_y )
E_result = E_bead(E_space, param_bead)
i_separatrix = []

for i in range(len(y0_list)):  # Go through the list of solutions to find specific ones.
    e = y0_list[i][2]
    if e == 0.00:
        i_separatrix.append(i)
    elif abs(e - U_bead(-np.pi, param_bead)) < 0.001:
        i_separatrix.append(i)
    

In [12]:
fig = make_subplots()  # (specs=[[{"secondary_y": True}]])
fig.update_layout(width=1000, height=800,
                  title=go.layout.Title(text=r"$\mathrm{Bead\ on\ a\ Loop\ Phase\ Space\ }\omega_0="+f"{omega0}$", x=0.5, xanchor="center"),
                  titlefont=dict(size=22),
                 xaxis_title=r"$\theta$", yaxis_title=r"$\omega$")

fig.add_trace(go.Contour(z=E_result, x=E_space_x, y=E_space_y, ncontours=20, line_smoothing=0.85, #contours_coloring='heatmap',
                        colorbar=dict(title='E', titleside='right',titlefont=dict(size=14,family='Times'))))

for i in i_separatrix:
    fig.add_trace(go.Scatter(x=solutions[i][0], y=solutions[i][1], name=rf"$E=0.0$", showlegend=show_leg, line=dict(color="#00FF50", width=2)))
    fig.add_trace(go.Scatter(x=solutions[i][0], y=solutions[i][1], name=rf"$E=0.0$", showlegend=show_leg, line=dict(color="#00FF50", width=2)))

fig.write_image("Phys615_HW8_figure4.pdf")
fig.show()

Figure 4: A contour plot of the energy of the bead on a loop system for $\omega_0=2.$, with the separatrix curves highlighted in green.